In [ ]:
import pickle
import time
import datetime
import os.path
import matplotlib.pyplot as plt
import numpy as np
import csv
import pathlib
import bisect


In [ ]:
start_date = datetime.datetime(2019, 1, 1, 0, 0,0)
end_date = datetime.datetime.now()
#end_date = datetime.datetime(2020, 3, 20, 0, 0,0)

total_days = (end_date-start_date).days + 1
print(total_days)

one_buy_shares_count=4

In [ ]:
def r2c(value):
    return float(int(value*100+0.5))/100

def fees(price,items_count):
    ret = price*items_count*0.5/100
    
    if items_count<100:
        ret+=1.2
    else:
        ret+=0.012*items_count
    
    return r2c(ret)


def scanFile(fileName) :
    ticker_name = os.path.basename(fileName).replace(".dat","")
    #print(ticker_name)

    arr = []
    f = open(fileName, 'rb')
    arr = pickle.load(f)
    f.close()

    cur_day = start_date
    
    start_index = bisect.bisect_left([r[0] for r in arr], cur_day.timestamp())
    
    last_c = 0
    total_buy=0.0
    buy_taxes=0.0
    shares=0

    for idx in range(start_index,len(arr)):
        
        row = arr[idx]
            
        date = datetime.datetime.fromtimestamp(row[0])
        o=float(row[1])
        h=float(row[2])
        l=float(row[3])
        c=float(row[4])
        
        last_c = c
        
        if date > end_date:
            break
       
        if date > cur_day:
            cur_day = date
            
        #if True:
        if date.weekday() == 0: #Monday
        #if date.weekday() == 4: #Friday
        #if date.day == 1:
        #if date.day == 15 and date.month == 1:
        #if date.day == 15 and date.month == 6:
            buy_price = (o+h+l+c)/4
            total_buy = total_buy + buy_price*one_buy_shares_count
            shares = shares + one_buy_shares_count
            buy_taxes = buy_taxes + fees(buy_price,one_buy_shares_count)
            
        gain = c*shares - fees(c,shares) - total_buy - buy_taxes
        
        index = (cur_day-start_date).days
        total_gain[index]=total_gain[index]+gain
        buy_all_shares[index]=buy_all_shares[index]+total_buy
        
    if total_buy != 0:
        gain = last_c*shares - fees(last_c,shares) - total_buy - buy_taxes
        print(f'{ticker_name}: gain={r2c(gain)}({r2c(gain/total_buy*100)}%) N={shares} value={r2c(last_c*shares)} total_buy={r2c(total_buy)} buy_taxes={r2c(buy_taxes)}')

In [ ]:
total_gain = np.zeros(total_days,dtype=float)
buy_all_shares = np.zeros(total_days,dtype=float)

#scanFile("bin/c.dat")
scanFile("bin/schd.dat")
scanFile("bin/spyd.dat")
#scanFile("bin/skm.dat")
scanFile("bin/lng.dat")
scanFile("bin/tan.dat")


In [ ]:
day_x=np.arange(0,len(total_gain),dtype=float)

del_arr=[]
for i in range(0,len(buy_all_shares)):
    if buy_all_shares[i] == 0.0:
        del_arr.append(i)
        
day_x = np.delete(day_x, del_arr, axis=0)
total_gain = np.delete(total_gain, del_arr, axis=0)
buy_all_shares = np.delete(buy_all_shares, del_arr, axis=0)

eff = total_gain / buy_all_shares * 100    

In [ ]:
days=[]
for d in day_x:
    days.append((start_date + datetime.timedelta(days=d)).strftime('%Y-%m-%d'))
    
print(days[0])
print(days[-1])
print(len(total_gain))
print(len(day_x))


In [ ]:
%matplotlib notebook
plt.plot(days,total_gain)
plt.show()


In [ ]:
%matplotlib notebook
fig, ax1 = plt.subplots()
color1 ='blue'
ax1.plot(days,total_gain, color = color1)
ax1.set_ylabel('Money $',color = color1);
ax1.tick_params(axis='y',color = color1);

color2 ='crimson'
ax2 = ax1.twinx();
ax2.plot(days,eff, color = color2)
ax2.set_ylabel('Eff. %', color = color2);
ax2.tick_params(axis='y',color = color2);
